# Config

In [ ]:
%run <Fundraising_Config>

In [ ]:
# Resources
bronze_lakehouse_name = "{BRONZE_LAKEHOUSE_NAME}"

# Configuration
source_name = "Salesforce_NPSP"
source_id = "4e5701ab-3c18-489d-93f7-0d62b78900f1"

# Feature flags
enable_create_tables = False

# Global functions

In [ ]:
def get_bronze_table(table_name: str):
    # if schemas (preview) enabled then use format {lakehouse}.dbo.{table_name}
    return spark.read.table(get_full_table_name(bronze_lakehouse_name, table_name))

# Metadata tables

## Watermark table

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, DateType, TimestampType, IntegerType, BooleanType, LongType
from pyspark.sql import SparkSession, DataFrame
import concurrent.futures
from datetime import datetime, timezone
import logging

WATERMARK_TABLE_NAME = "Metadata_Watermark"

class SalesforceNpspBronzeModel(BaseDataModel):
    def get_entities(self) -> list[tuple[StructType, str, bool]]:
        return [
            (SalesforceNpspBronzeModel.get_watermark_schema(), WATERMARK_TABLE_NAME, False)
        ]
        
    @staticmethod
    def get_watermark_schema() -> StructType:
        return StructType([
            StructField("ObjectName", StringType(), False),
            StructField("LastWatermark", TimestampType(), False),
            StructField("UpdatedAt", TimestampType(), False)
        ])


# Initialize the model and create the tables
if enable_create_tables:
    spark = SparkSession.builder.getOrCreate()

    logging.info("Creating Salesforce NPSP bronze data model entities.")

    model = SalesforceNpspBronzeModel(spark=spark, lakehouse=bronze_lakehouse_name)
    model.create_entities()

    logging.info("All tables created.")